In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
cwd = os.path.join(os.getcwd())

dir_path = os.path.join(cwd, 'raw_datasets/era5')

In [3]:
minyear = 1950
maxyear = 2023

lon_Barter_Island = -143.75
lat_Barter_Island = 70.0

lon_offshore = -143.75
lat_offshore = 70.25

In [4]:
def mask_data_spatial(ds, lon, lat):
    
    spatial_mask = (ds.longitude == lon) * (ds.latitude == lat)

    return ds.where(spatial_mask, drop=True)

In [5]:
locations = []


def convert_to_csv(var_name, minyear, maxyear, lon, lat, save_path=None, lat_offshore=70.25, lat_Barter_Island=70.0, save=False):
    
    time = []
    data = []

    years_with_data = np.arange(minyear, maxyear+1, 1)

    for year in years_with_data:
        
        fp = os.path.join(dir_path, var_name + "_" + str(year) + ".nc")
        
        ds = xr.open_dataset(fp)
                
        masked_ds = mask_data_spatial(ds, lon, lat).squeeze(drop=True)
        
        colname = list(ds.keys())[0]
        
        ds.close()

        time_year = list(masked_ds.time.values)
       
        time = time + time_year
        
        data += list(masked_ds[colname].values)
        
        masked_ds.close()
            
    df = pd.DataFrame({"time":time})

    if not save_path:
        save_path = var_name
    
    df[save_path] = data
    
    if lat == lat_offshore:
        locations.append((save_path, lon_offshore, lat_offshore, 'offshore'))
    else:
        locations.append((save_path, lon_Barter_Island, lat_Barter_Island, 'onshore'))
    
    if save:
        df.to_csv(os.path.join("ts_datasets/", save_path + ".csv"))
    
    print(f"succesful for variable {save_path}")
    
    return df

In [6]:
# offshore_vars = [
#     "sea_ice_cover", 
#     "sea_surface_temperature", 
#     "10m_u_component_of_wind",
#     "10m_v_component_of_wind"
#     "soil_temperature_level_1_offs",
#     "soil_temperature_level_2_offs",
#     "soil_temperature_level_3_offs",
#     "soil_temperature_level_4_offs"
# ]

dfs = [
convert_to_csv('10m_u_component_of_wind', minyear, maxyear, lon_offshore, lat_offshore),
convert_to_csv('10m_v_component_of_wind', minyear, maxyear, lon_offshore, lat_offshore),
convert_to_csv('2m_temperature', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('mean_surface_latent_heat_flux', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('mean_surface_net_long_wave_radiation_flux', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('mean_surface_net_short_wave_radiation_flux', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('sea_ice_cover', minyear, maxyear, lon_offshore, lat_offshore),
convert_to_csv('sea_surface_temperature', minyear, maxyear, lon_offshore, lat_offshore),
convert_to_csv('snow_depth', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_1', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_2', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_3', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_4', minyear, maxyear, lon_Barter_Island, lat_Barter_Island),
convert_to_csv('soil_temperature_level_1', minyear, maxyear, lon_offshore, lat_offshore, save_path='soil_temperature_level_1_offs'),
convert_to_csv('soil_temperature_level_2', minyear, maxyear, lon_offshore, lat_offshore, save_path='soil_temperature_level_2_offs'),
convert_to_csv('soil_temperature_level_3', minyear, maxyear, lon_offshore, lat_offshore, save_path='soil_temperature_level_3_offs'),
convert_to_csv('soil_temperature_level_4', minyear, maxyear, lon_offshore, lat_offshore, save_path='soil_temperature_level_4_offs'),
]

succesful for variable 10m_u_component_of_wind
succesful for variable 10m_v_component_of_wind
succesful for variable 2m_temperature
succesful for variable mean_surface_latent_heat_flux
succesful for variable mean_surface_net_long_wave_radiation_flux
succesful for variable mean_surface_net_short_wave_radiation_flux
succesful for variable sea_ice_cover
succesful for variable sea_surface_temperature
succesful for variable snow_depth
succesful for variable soil_temperature_level_1
succesful for variable soil_temperature_level_2
succesful for variable soil_temperature_level_3
succesful for variable soil_temperature_level_4
succesful for variable soil_temperature_level_1_offs
succesful for variable soil_temperature_level_2_offs
succesful for variable soil_temperature_level_3_offs
succesful for variable soil_temperature_level_4_offs


In [11]:
print(dfs[0])

                      time  10m_u_component_of_wind
0      1950-01-01 00:00:00                 9.184960
1      1950-01-01 01:00:00                 8.612046
2      1950-01-01 02:00:00                 8.351060
3      1950-01-01 03:00:00                 7.876539
4      1950-01-01 04:00:00                 7.445283
...                    ...                      ...
648667 2023-12-31 19:00:00                -1.526860
648668 2023-12-31 20:00:00                -1.611450
648669 2023-12-31 21:00:00                -1.563535
648670 2023-12-31 22:00:00                -2.078762
648671 2023-12-31 23:00:00                -2.100058

[648672 rows x 2 columns]


In [7]:
df_loc = pd.DataFrame(data=locations, columns=['var', 'lon', 'lat', 'note'])
df_loc.to_csv(os.path.join("ts_datasets/", "variable_locations.txt"))

In [8]:
time = pd.read_csv(os.path.join("ts_datasets/", "2m_temperature.csv"))["time"].values

total_df = pd.DataFrame(time, columns=["time"])
total_df.head()

,time
0,1950-01-01 00:00:00
1,1950-01-01 01:00:00
2,1950-01-01 02:00:00
3,1950-01-01 03:00:00
4,1950-01-01 04:00:00


In [9]:
dir_list = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_temperature",
    "mean_surface_latent_heat_flux",
    "mean_surface_net_long_wave_radiation_flux",
    "mean_surface_net_short_wave_radiation_flux",
    "sea_ice_cover",
    "sea_surface_temperature",
    "snow_depth",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4",
    "soil_temperature_level_1_offs",
    "soil_temperature_level_2_offs",
    "soil_temperature_level_3_offs",
    "soil_temperature_level_4_offs",
]

In [14]:
for dir, df in zip(dir_list, dfs):
    total_df[dir] = df[dir].values
    
    

In [ ]:
# for dir in dir_list:
    
#     fpath = os.path.join("ts_datasets", dir + ".csv")
    
#     with open(fpath) as f:
        
#         array = pd.read_csv(f).iloc[:,2]
        
#         total_df[dir] = array

In [15]:
total_df.to_csv(os.path.join("ts_datasets/", "era5.csv"))